In [6]:
from atj_algotrading.backtester import Backtester, get_ohlc_history 
from datetime import datetime, timedelta
import MetaTrader5 as mt5

In [4]:
# connect to MT5
mt5.initialize()

True

In [10]:
symbol = 'BTCUSD'
timeframe = mt5.TIMEFRAME_D1
start_dt = datetime.now() - timedelta(days=365 * 10)
end_dt = datetime.now()

ohlc_df = get_ohlc_history(symbol, timeframe, start_dt, end_dt)
ohlc_df['month'] = ohlc_df['time'].dt.month
ohlc_df

,time,open,high,low,close,month
0,2014-10-05,301.05,320.60,292.30,312.88,10
1,2014-10-06,312.88,339.99,296.44,322.00,10
2,2014-10-07,321.94,334.49,310.11,327.00,10
3,2014-10-08,327.00,350.00,320.59,348.22,10
4,2014-10-09,348.00,378.75,340.02,357.90,10
...,...,...,...,...,...,...
3060,2024-09-26,63431.72,65848.12,62647.68,64732.32,9
3061,2024-09-27,64628.84,66498.35,64628.84,65808.39,9
3062,2024-09-28,65787.34,66259.28,65415.45,65653.40,9
3063,2024-09-29,65629.62,66082.10,65422.87,65817.93,9


In [17]:
# create trade logic
def on_bar(data, trades, orders):
    volume = round(10000 / data['open'], 2)
    
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    
    # entry signal
    if data['month'] in [10] and not num_open_trades:
        orders.open_trade(symbol, volume, 'buy')
        
    # exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]
        
        if data['month'] in [11]:
            orders.close_trade(trade)

In [18]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -20

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(ohlc_df)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,BTCUSD,buy,33.22,2014-10-05 00:00:00,301.05,2014-11-02 00:00:00,321.86,0,0,None,691.31,-664.4,26.91,26.91,10026.91
1,closed,BTCUSD,buy,42.9,2015-10-01 00:00:00,233.08,2015-11-01 00:00:00,308.1,0,0,None,3218.36,-858.0,2360.36,2387.27,12387.27
2,closed,BTCUSD,buy,16.44,2016-10-02 00:00:00,608.29,2016-11-01 00:00:00,689.0,0,0,None,1326.87,-328.8,998.07,3385.34,13385.34
3,closed,BTCUSD,buy,2.31,2017-10-02 00:00:00,4325.37,2017-11-01 00:00:00,6348.22,0,0,None,4672.78,-46.2,4626.58,8011.92,18011.92
4,closed,BTCUSD,buy,1.53,2018-10-01 00:00:00,6536.11,2018-11-01 00:00:00,6287.58,0,0,None,-380.25,-30.6,-410.85,7601.07,17601.07
5,closed,BTCUSD,buy,1.22,2019-10-01 00:00:00,8214.25,2019-11-01 00:00:00,9115.93,0,0,None,1100.05,-24.4,1075.65,8676.72,18676.72
6,closed,BTCUSD,buy,0.94,2020-10-01 00:00:00,10692.75,2020-11-02 00:00:00,13812.25,0,0,None,2932.33,-18.8,2913.53,11590.25,21590.25
7,closed,BTCUSD,buy,0.23,2021-10-01 00:00:00,43343.99,2021-11-01 00:00:00,60858.01,0,0,None,4028.22,-4.6,4023.62,15613.87,25613.87
8,closed,BTCUSD,buy,0.52,2022-10-01 00:00:00,19382.38,2022-11-01 00:00:00,20363.87,0,0,None,510.37,-10.4,499.97,16113.84,26113.84
9,closed,BTCUSD,buy,0.37,2023-10-01 00:00:00,27032.48,2023-11-01 00:00:00,34603.42,0,0,None,2801.25,-7.4,2793.85,18907.69,28907.69


In [19]:
pnl_chart = bt.plot_pnl()
pnl_chart